In [1]:
# import cv2
# import numpy as np

# cap =  cv2.VideoCapture(0, cv2.CAP_DSHOW)
# ref_img = False 

# while cap.isOpened():
    
#     success, frame_img = cap.read()
#     # frame = cv2.rotate(frame, cv2.ROTATE_180)

#     if not success:
#         print("Ignoring empty camera frame.")
#         break

#     if not ref_img:    
#         cv2.imshow('Pressed Key Frame', frame_img)
        
#         if cv2.waitKey(1) & 0xFF == ord('s'):
#             break

#     cv2.waitKey(1)
#     if cv2.getWindowProperty('Pressed Key Frame', cv2.WND_PROP_VISIBLE) < 1:
#         break
    
# cap.release()
# cv2.destroyAllWindows()

In [2]:
# # Initialize blank images for color exploration and selection
# color_explore = np.zeros((150, 150, 3), dtype=np.uint8)
# color_selected = np.zeros((150, 150, 3), dtype=np.uint8)

# # Mouse callback function for showing color under cursor and selected color
# def show_color(event, x, y, flags, param):
    
#     # Extract color components from the image at cursor position
#     B, G, R = img[y, x]

#     # Update color_explore with the color under cursor
#     color_explore[:] = (B, G, R)

#     # If left mouse button pressed, update color_selected and print the color values
#     if event == cv2.EVENT_LBUTTONDOWN:
#         color_selected[:] = (B, G, R)
#         black = (R, G, B)
#         print(black)

# # Create windows for color exploration and selected color display
# cv2.namedWindow('color_explore')
# cv2.resizeWindow('color_explore', 50, 50)

# cv2.namedWindow('color_selected')
# cv2.resizeWindow('color_selected', 50, 50)

# # Create window for displaying the sample image
# cv2.namedWindow('image')

# # Path to the sample image
# img = frame_img

# # Assign the mouse callback function to the 'image' window
# cv2.setMouseCallback('image', show_color)

# # Main loop for live update of the color exploration and selection
# while True:
#     cv2.imshow('image', img)
#     cv2.imshow('color_explore', color_explore)
#     cv2.imshow('color_selected', color_selected)

#     # Break the loop if 'Esc' key is pressed
#     if cv2.waitKey(1) & 0xFF == 27:
#         break

# # Clean up windows
# cv2.destroyAllWindows()


In [8]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import time


# create hex_color_picker

def hex_to_bgr(hex_color):
    hex_color = hex_color.lstrip('#')
    lv = len(hex_color)
    return tuple(int(hex_color[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))[::-1]

def apply_threshold(roi, bgr_color, threshold):
    """Apply color thresholding to isolate specific color ranges in the ROI."""
    
    lower_bound = np.array([max(0, bgr_color[0] - threshold), max(0, bgr_color[1] - threshold), max(0, bgr_color[2] - threshold)])
    upper_bound = np.array([min(255, bgr_color[0] + threshold), min(255, bgr_color[1] + threshold), min(255, bgr_color[2] + threshold)])
    mask = cv2.inRange(roi, lower_bound, upper_bound)
    
    return mask

def find_clusters(mask):
    """Find clusters in the mask using DBSCAN."""
    
    y_coord, x_coord = np.where(mask != 0)
    if len(y_coord) == 0:
        return {}  # Return an empty dict if no points found
    
    coord_array = np.stack((y_coord, x_coord), axis=-1)
    sorted_array = coord_array[coord_array[:, 1].argsort()]
    dbscan = DBSCAN(eps=5, min_samples=10)
    clusters = dbscan.fit_predict(sorted_array)

    cluster_dict = {}
    for point, cluster_idx in zip(sorted_array, clusters):
        if cluster_idx != -1:
            cluster_dict.setdefault(cluster_idx, []).append(point.tolist())
            
    return cluster_dict

def filter_noise_clusters(cluster_dict, size_threshold):
    """
    Filters clusters based on a minimum size threshold.

    Parameters:
    - cluster_dict (dict): A dictionary where each key represents a cluster index,
      and the value is a list of points belonging to that cluster.
    - size_threshold (int): The minimum number of points a cluster must have to be included.

    Returns:
    - dict: A new dictionary containing only the clusters that meet the size threshold.
    """
    
    filtered_clusters = {}
    for key, points in cluster_dict.items():
        if len(points) > size_threshold:
            filtered_clusters[key] = points
            
    return filtered_clusters

def generate_error_bounds_for_clusters(cluster_dict_black, cluster_dict_white, initial_threshold=6):
    """
    Generates error bounds for black and white clusters based on their counts and an initial threshold.

    Parameters:
    - cluster_dict_black (dict): The black clusters dictionary.
    - cluster_dict_white (dict): The white clusters dictionary.
    - initial_threshold (int): The initial threshold for error calculation.

    Returns:
    - tuple: Contains two tuples for black and white error bounds, each with a list for lower and upper bounds.
    """
    error_lower_bound_black = [initial_threshold for _ in range(len(cluster_dict_black))]
    error_upper_bound_black = [initial_threshold + 12 for _ in range(len(cluster_dict_black))]
    
    error_lower_bound_white = [initial_threshold for _ in range(len(cluster_dict_white))]
    error_upper_bound_white = [initial_threshold + 12 for _ in range(len(cluster_dict_white))]
    print("cluster_dict_white", cluster_dict_white)

    black_error_bounds = (error_lower_bound_black, error_upper_bound_black)
    white_error_bounds = (error_lower_bound_white, error_upper_bound_white)
    return black_error_bounds, white_error_bounds


def calibrate_error_bounds(error_keys, error_bounds):
    """
    Adjusts error bounds based on the presence of keys in the error_keys list. If error_keys is not empty,
    increments the bounds for those keys. Otherwise, indicates that calibration is done.

    Parameters:
    - error_keys (list): A list of indices corresponding to clusters that met error criteria.
    - error_bounds (tuple): A tuple containing two lists (error_lower_bound, error_upper_bound) representing the current error bounds for filtering.

    Returns:
    - tuple: The updated error_bounds tuple after adjustment.
    """
    error_lower_bound, error_upper_bound = error_bounds
    if error_keys:
        for _, value in enumerate(error_keys):
            error_lower_bound[value] += 1
            error_upper_bound[value] += 1
        updated_bounds = (error_lower_bound, error_upper_bound)
    else:
        print("calibration_done:", error_lower_bound, error_upper_bound)
        # each value in error_lower_bound + something to notget other keys  
        updated_bounds = error_bounds  # No change if calibration done

    return updated_bounds


def reference_frame(frame, mask_bound, bgr_color_1, bgr_color_2, threshold=40):
    x1, y1, x2, y2 = mask_bound
    roi = frame[y1:y2, x1:x2]

    # black
    # bgr_color_1 = hex_to_bgr(hex_color_1)  # "#C8CE7B"
    mask_1 = apply_threshold(roi, bgr_color_1, threshold)
    roi[mask_1 != 0] = [255, 0, 0]
    cluster_dict_1 = find_clusters(mask_1)

    # white
    # bgr_color_2 = hex_to_bgr(hex_color_2)  # "#C8CE7B"
    mask_2 = apply_threshold(roi, bgr_color_2, threshold)
    roi[mask_2 != 0] = [0, 0, 255]
    cluster_dict_2 = find_clusters(mask_2)

    # cluster_dict_1 = filter_noise_clusters(cluster_dict_1, 100)
    cluster_dict_2 = filter_noise_clusters(cluster_dict_2, 100)

    ###### Check if this improves the latency otherwise delete it #########
    for cluster_idx in cluster_dict_1:
        cluster_dict_1[cluster_idx] = np.array(cluster_dict_1[cluster_idx])

    for cluster_idx in cluster_dict_2:
        cluster_dict_2[cluster_idx] = np.array(cluster_dict_2[cluster_idx])
    
    #######################################################################

    return roi, cluster_dict_1, cluster_dict_2
    

def filter_keys(cluster_dict, roi, inf_roi, error_bounds):
    """
    Calculates error percentages for clusters and filters keys based on error bounds.

    Parameters:
    - cluster_dict (dict): Clusters to analyze, where each key is a cluster index, and the value is a list of points.
    - roi (numpy.ndarray): The reference region of interest from the original frame.
    - inf_roi (numpy.ndarray): The inference region of interest from the compared frame.
    - error_bounds (tuple): A tuple containing two lists, the first for lower bounds and the 
      second for upper bounds of error percentages for filtering. Each list's length should match the number of clusters.

    Returns:
    - tuple: (error_keys, error_percentages)
        - error_keys (list): The keys of clusters that fall within the specified error bounds.
        - error_percentages (list): The error percentages of all clusters.
    """
    error_keys = []
    error_percentages = []
    error_lower_bound, error_upper_bound = error_bounds

    for index, (key, cluster) in enumerate(cluster_dict.items()):
        error_count = sum(1 for row_ref, col_ref in cluster if not np.array_equal(roi[row_ref, col_ref], inf_roi[row_ref, col_ref]))
        total_comparisons = len(cluster)

        error_percentage = (error_count / total_comparisons) * 100 if total_comparisons > 0 else 0
        error_percentages.append(error_percentage)

        if total_comparisons > 0 and error_lower_bound[index] < error_percentage < error_upper_bound[index]:
            error_keys.append(key)

    return error_keys


def inference_frame(inf_frame, mask_bound, bgr_color_1, bgr_color_2, cluster_dict_1, cluster_dict_2, roi, threshold=40, error_bound_1=(), error_bound_2=()):
    x1, y1, x2, y2 = mask_bound
    inf_roi = inf_frame[y1:y2, x1:x2]

    # black
    # bgr_color_1 = hex_to_bgr(hex_color_1)  # "#C8CE7B"
    mask_1 = apply_threshold(inf_roi, bgr_color_1, threshold)
    inf_roi[mask_1 != 0] = [255, 0, 0]

    # white
    # bgr_color_2 = hex_to_bgr(hex_color_2)  # "#C8CE7B"
    mask_2 = apply_threshold(inf_roi, bgr_color_2, threshold)
    inf_roi[mask_2 != 0] = [255, 0, 255]

    black_error_keys = filter_keys(cluster_dict_1, roi, inf_roi, error_bound_1)
    white_error_keys = filter_keys(cluster_dict_2, roi, inf_roi, error_bound_2)

    return inf_roi, black_error_keys, white_error_keys

def inference_frame_calibration(inf_frame, mask_bound, bgr_color_1, bgr_color_2, cluster_dict_1, cluster_dict_2, roi,
                                threshold=40, error_bounds_1=(), error_bounds_2=(), initial_state=True):
    x1, y1, x2, y2 = mask_bound
    inf_roi = inf_frame[y1:y2, x1:x2]

    # black
    # bgr_color_1 = hex_to_bgr(hex_color_1)  # "#C8CE7B"
    mask_1 = apply_threshold(inf_roi, bgr_color_1, threshold)
    inf_roi[mask_1 != 0] = [255, 0, 0]

    # white
    # bgr_color_2 = hex_to_bgr(hex_color_2)  # "#C8CE7B"
    mask_2 = apply_threshold(inf_roi, bgr_color_2, threshold)
    inf_roi[mask_2 != 0] = [255, 0, 255]

    if (initial_state):
        error_bounds_1, error_bounds_2 = generate_error_bounds_for_clusters(cluster_dict_1, cluster_dict_2, initial_threshold=6)
        print(error_bounds_1, error_bounds_2)

    black_error_keys = filter_keys(cluster_dict_1, roi, inf_roi, error_bounds_1)
    white_error_keys = filter_keys(cluster_dict_2, roi, inf_roi, error_bounds_2)

    black_error_bounds = calibrate_error_bounds(black_error_keys, error_bounds_1)
    white_error_bounds = calibrate_error_bounds(white_error_keys, error_bounds_2)

    return inf_roi, black_error_keys, white_error_keys, black_error_bounds, white_error_bounds


def highlight_clusters(frame_roi, cluster_dict, error_keys, color=(0, 0, 255)):
    """
    Highlights specified clusters in a region of interest by changing their pixel colors.

    Parameters:
    - frame_roi (numpy.ndarray): The region of interest from the frame where clusters are to be highlighted.
    - cluster_dict (dict): A dictionary containing clusters with their points. Each key in the dictionary
      represents a cluster index, and the value is a list of points (row, column pairs) belonging to that cluster.
    - error_keys (list): A list of keys indicating which clusters in the cluster_dict should be highlighted.
    - color (tuple): The BGR color value to use for highlighting. Default is red (0, 0, 255).

    Returns:
    - numpy.ndarray: The modified region of interest with specified clusters highlighted.
    """
    for key in error_keys:
        if key in cluster_dict:  # Check if the key exists in the cluster dictionary
            for row, col in cluster_dict[key]:
                frame_roi[row, col] = color
    return frame_roi
    

white_keys = ['C', 'D', 'E', 'F', 'G', 'A', 'B']
black_keys = ['C#', 'D#', 'F#', 'G#', 'A#']

def encode_to_scale(values, scale):
    encoded_notes = []
    scale_length = len(scale)
    for value in values:
        # Map each value to a note in the scale
        note = scale[value % scale_length]
        encoded_notes.append(note)
    return encoded_notes

In [7]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
ref_img = False
count = 0
calibration_frames = 150
BGR_color_1 = (236, 147, 195)
BGR_color_2 = (214, 224, 155)
print("starting loop", cap)
while cap.isOpened():
    if count < calibration_frames:
        count += 1
    success, frame_img = cap.read()

    if not success:
        print("Ignoring empty camera frame.")
        break

    if not ref_img:
        if count == 100:
            start = time.perf_counter()
            
            mask_bound = (0, 265, 640, 452)
            roi, cluster_dict_1, cluster_dict_2 = reference_frame(frame_img, mask_bound, BGR_color_1, BGR_color_2)

            ref_img = True
            first_state = True
            frame_img = roi
            
            end = time.perf_counter() - start
            print('{:.6f}s for the calculation'.format(end))

    elif (ref_img):
        if count < calibration_frames:
            print("calibrating...")
            if(first_state):
                
                frame_roi, black_error_keys, white_error_keys, black_error_bounds, white_error_bounds = inference_frame_calibration(
                    frame_img, mask_bound, BGR_color_1, BGR_color_2, cluster_dict_1, cluster_dict_2,
                    roi, threshold=40)
                
                first_state = False

            else:
                frame_roi, black_error_keys, white_error_keys, black_error_bounds, white_error_bounds = inference_frame_calibration(
                    frame_img, mask_bound, BGR_color_1, BGR_color_2, cluster_dict_1, cluster_dict_2,
                    roi, threshold=40, error_bounds_1=black_error_bounds, error_bounds_2=white_error_bounds,
                    initial_state=False)

                frame_roi = highlight_clusters(frame_roi, cluster_dict_1, black_error_keys, color=(0, 0, 255))
                frame_roi = highlight_clusters(frame_roi, cluster_dict_2, white_error_keys, color=(0, 255, 255))

        else:
            frame_roi, black_error_keys, white_error_keys = inference_frame(frame_img, mask_bound, BGR_color_1, BGR_color_2,
                cluster_dict_1, cluster_dict_2, roi, threshold=40, 
                error_bound_1=black_error_bounds, error_bound_2=white_error_bounds)
            
            encoded_notes_white = encode_to_scale(white_error_keys, white_keys)
            encoded_notes_black = encode_to_scale(black_error_keys, black_keys)
            
            all_notes = encoded_notes_white + encoded_notes_black
            if (all_notes):
                print(all_notes)
                
            frame_roi = highlight_clusters(frame_roi, cluster_dict_1, black_error_keys, color=(0, 0, 255))
            frame_roi = highlight_clusters(frame_roi, cluster_dict_2, white_error_keys, color=(0, 255, 255))

        cv2.waitKey(1)
        cv2.imshow('Pressed Key Frame', frame_roi)


cap.release()
cv2.destroyAllWindows()

starting loop < cv2.VideoCapture 000001E1C0C0D450>
0.050970s for the calculation
calibrating...
cluster_dict_white {8: array([[ 68,  89],
       [ 67,  89],
       [114,  89],
       [ 78,  89],
       [109,  89],
       [115,  89],
       [ 66,  89],
       [ 89,  89],
       [ 63,  89],
       [ 70,  89],
       [ 69,  89],
       [ 72,  89],
       [ 64,  89],
       [112,  89],
       [124,  89],
       [ 73,  89],
       [ 74,  89],
       [ 77,  89],
       [113,  89],
       [ 65,  89],
       [111,  89],
       [ 75,  89],
       [117,  89],
       [110,  89],
       [ 76,  89],
       [116,  89],
       [108,  89],
       [ 82,  89],
       [125,  89],
       [ 60,  89],
       [ 97,  89],
       [ 79,  89],
       [128,  89],
       [ 95,  89],
       [ 94,  89],
       [ 83,  89],
       [119,  89],
       [121,  89],
       [ 84,  89],
       [ 93,  89],
       [ 85,  89],
       [ 92,  89],
       [ 86,  89],
       [120,  89],
       [ 87,  89],
       [ 91,  89],
       